In [102]:
import pickle
import numpy as np
import xgboost as xgb
import glob
import pandas as pd

fl = glob.glob('../features/*.pkl')
for l in fl:
    print("'{}',".format(l))

'../features/vgg16_1_feat.pkl',
'../features/cnn_1_feat.pkl',
'../features/resnet_aug1_feat.pkl',
'../features/incept_aug1_feat.pkl',
'../features/other_feat.pkl',
'../features/incept_aug2_feat.pkl',
'../features/cnn_1_aug1_feat.pkl',
'../features/resnet_1_feat.pkl',
'../features/cnn_3_aug1_feat.pkl',
'../features/cnn_2_aug1_feat.pkl',
'../features/cnn_2_feat.pkl',
'../features/cnn_4_aug1_feat.pkl',
'../features/cnn_4_aug1_feat_add_early.pkl',
'../features/cnn_5_aug1_feat.pkl',
'../features/incept_1_feat.pkl',
'../features/resnet_aug3_feat.pkl',
'../features/incept_aug3_feat.pkl',
'../features/vgg_aug1_feat.pkl',
'../features/resnet_aug2_feat.pkl',


In [103]:
choose_fl = [
#'../features/cnn_1_feat.pkl',
#'../features/resnet_aug1_feat.pkl',
#'../features/incept_aug2_feat.pkl',
#'../features/cnn_3_aug1_feat.pkl',
#'../features/cnn_2_feat.pkl',
#'../features/cnn_4_aug1_feat_add_early.pkl',
#'../features/cnn_5_aug1_feat.pkl',
#'../features/incept_1_feat.pkl',
#'../features/incept_aug3_feat.pkl',
#'../features/vgg_aug1_feat.pkl',
'../features/vgg16_1_feat.pkl',
'../features/incept_aug1_feat.pkl',
'../features/other_feat.pkl',
'../features/cnn_1_aug1_feat.pkl',
'../features/resnet_1_feat.pkl',
'../features/cnn_2_aug1_feat.pkl',
'../features/cnn_4_aug1_feat.pkl',
'../features/resnet_aug3_feat.pkl',
'../features/resnet_aug2_feat.pkl',
  
]

def load_x_y():
    train_x,test_x=[],[]
    for f in choose_fl:
        print(f)
        with open(f,'rb') as fin:
            a,b = pickle.load(fin)
            train_x.append(a)
            test_x.append(b)
    train_x = np.hstack(train_x)
    test_x = np.hstack(test_x)
    train_df = pd.read_json('../input/train.json')
    y = train_df.is_iceberg.values
    return train_x,test_x,y

train_x,test_x,train_y = load_x_y()
print(train_x.shape,test_x.shape,train_y.shape)

../features/vgg16_1_feat.pkl
../features/incept_aug1_feat.pkl
../features/other_feat.pkl
../features/cnn_1_aug1_feat.pkl
../features/resnet_1_feat.pkl
../features/cnn_2_aug1_feat.pkl
../features/cnn_4_aug1_feat.pkl
../features/resnet_aug3_feat.pkl
../features/resnet_aug2_feat.pkl
(1604, 19) (8424, 19) (1604,)


In [104]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
def cv_test(k_cnt=3,rnd=42,save_flag=False):
    kf = StratifiedKFold(n_splits=k_cnt, shuffle=True, random_state=rnd*2)
    w_test_pred = None
    r_val_loss = 0
    test_pred = None
    val_loss = 0
    val_loss_list = []
    for train_index, test_index in kf.split(train_x,train_y):
        X_train, X_test = train_x[train_index], train_x[test_index]
        y_train, y_test = train_y[train_index], train_y[test_index]
        params = {
                'colsample_bytree': 0.75,
                'colsample_bylevel':0.95,
                'gamma':0.2,
                'subsample': 0.9,
                'eta': 0.07,
                'max_depth': 3,
                'eval_metric':'logloss',
                'objective':'binary:logistic',
                'scale_pos_weight': 1.0
                }
        
        # def mat
        d_train = xgb.DMatrix(X_train, y_train)
        d_valid = xgb.DMatrix(X_test, y_test)
        d_test = xgb.DMatrix(test_x)
        
        watchlist = [(d_train, 'train'), (d_valid, 'valid')]
        # train model
        m = xgb.train(params, d_train, 500, watchlist, 
                        early_stopping_rounds=30,
                        verbose_eval=100)
        
        # cal val loss
        curr_val_loss = log_loss(y_test,m.predict(d_valid))
        val_loss += curr_val_loss/k_cnt
        val_loss_list.append(curr_val_loss)
        curr_r_val_loss = 1.0 / curr_val_loss
        r_val_loss += curr_r_val_loss
        
        if test_pred is None:
            test_pred = m.predict(d_test)
            w_test_pred = m.predict(d_test)*curr_r_val_loss
        else:
            curr_pred = m.predict(d_test)
            test_pred += curr_pred
            w_test_pred = w_test_pred + curr_pred*curr_r_val_loss

    # avg
    test_pred = test_pred / k_cnt
    w_test_pred = w_test_pred / r_val_loss
    
    # train log loss
    print('local average valid loss',val_loss,'val loss std',np.std(val_loss_list))
    if save_flag:
        test_df=pd.read_json('../input/test.json')
        submission = pd.DataFrame()
        submission['id']=test_df['id']
        submission['is_iceberg']=test_pred
        print(submission.head())
        submission.to_csv('../results/xgb_sub_fold_{}_rnd_{}.csv'.format(k_cnt,rnd), index=False)
        
        submission = pd.DataFrame()
        submission['id']=test_df['id']
        submission['is_iceberg']=w_test_pred
        print(submission.head())
        submission.to_csv('../results/weighted_xgb_sub_fold_{}_rnd_{}.csv'.format(k_cnt,rnd), index=False)

    
print('def done')

def done


In [105]:
cv_test(5,42,True)

[0]	train-logloss:0.639003	valid-logloss:0.643103
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
[100]	train-logloss:0.085426	valid-logloss:0.16726
Stopping. Best iteration:
[159]	train-logloss:0.059304	valid-logloss:0.158494

[0]	train-logloss:0.640624	valid-logloss:0.642576
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
[100]	train-logloss:0.094006	valid-logloss:0.139332
[200]	train-logloss:0.05262	valid-logloss:0.127287
Stopping. Best iteration:
[217]	train-logloss:0.047615	valid-logloss:0.124558

[0]	train-logloss:0.64019	valid-logloss:0.64249
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
[100]	train-logloss:0.0877	valid-logloss:0.181453
Stopping. Best iteration:
[128]	train

In [106]:
cv_test(5,233,True)

[0]	train-logloss:0.64014	valid-logloss:0.640318
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
[100]	train-logloss:0.088416	valid-logloss:0.149747
Stopping. Best iteration:
[131]	train-logloss:0.071451	valid-logloss:0.146607

[0]	train-logloss:0.640654	valid-logloss:0.639231
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
[100]	train-logloss:0.101951	valid-logloss:0.102936
[200]	train-logloss:0.056535	valid-logloss:0.08979
Stopping. Best iteration:
[223]	train-logloss:0.050506	valid-logloss:0.087753

[0]	train-logloss:0.639738	valid-logloss:0.642165
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
[100]	train-logloss:0.090744	valid-logloss:0.149501
Stopping. Best iteration:
[143]	t